## Pluralsight Course : Building and Deploying RAG in Production

#### Demo: Basic RAG

In [ ]:
# install the required libraries
!pip install -qqq llama-index llama-index-llms-openai llama-index-vector-stores-chroma


In [ ]:
import os 
# set API Key
os.environ["OPENAI_API_KEY"] = "<YOU_API_KEY>"

### Define Embedding and LLM Model

In [3]:
from llama_index.embeddings.openai import OpenAIEmbedding
# define embeding model 
embed_model = OpenAIEmbedding()

In [4]:
from llama_index.llms.openai import OpenAI
# define LLM model
llm = OpenAI(model="gpt-4o", temperature=0)

In [5]:
from llama_index.core import Settings

# setting embedding model and llm model globally
Settings.embed_model = embed_model
Settings.llm = llm

### Ingestion Pipeline

In [6]:
# set chunk size
Settings.chunk_size = 1024

In [7]:
# import libraries 
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import TokenTextSplitter

# load
documents = SimpleDirectoryReader("data").load_data()


In [8]:
# define chunking strategy
text_splitter = TokenTextSplitter()

In [9]:
# define vector database and store 
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

# create local in-memory client
chroma_client = chromadb.EphemeralClient()
# create a collection
chroma_collection = chroma_client.create_collection("ps-foo-rag", get_or_create=True)
# define the vector store using the collection
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [10]:
# define ingestion pipeline
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        embed_model,
    ],
    vector_store=vector_store,
)


In [11]:
# run the ingestion pipeline
nodes = pipeline.run(documents=documents)
print(f"number of nodes or chunks : {len(nodes)}")

number of nodes or chunks : 35


### RAG Pipeline

In [12]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model,
)

In [13]:
# create semantic query engine 
vector_query_engine = vector_index.as_query_engine()

In [14]:
# query 
response = vector_query_engine.query("who is the CEO of the company ?")
print(response)

The CEO of the company is Bar.


***Sample Queries to test***

- When did Bar started his entrepreneurial journey ?
- Who are the co-founders of the company ?
- What is the Bar's AI vision?
- Who is the CEO of company Foo? and what are the other companies he has started earlier.
